# Random Forest Training - DEFRA All Stations (6 Pollutants)

This notebook trains Random Forest models for **all station-pollutant combinations** in the DEFRA dataset, limited to the 6 regulatory pollutants that match LAQN for direct comparison.

## What this notebook does

1. Load prepared data from `ml_prep_all` output.
2. Train separate RF models for each station-pollutant target.
3. Use checkpoint saving for overnight training safety.
4. Evaluate using RMSE, MAE, R² for each target.
5. Aggregate results by pollutant type.
6. Save all models and results.

## Why train separate models?

From earlier experiments, `MultiOutputRegressor` with 100+ targets causes memory issues on 8GB RAM machines. Training separate models:

| Approach | Memory | Time | Flexibility |
| --- | --- | --- | --- |
| MultiOutputRegressor | High (all at once) | Faster total | Less control |
| Separate models | Low (one at a time) | Slower total | Can resume from checkpoint |

The separate approach is safer for overnight training and allows resuming if something goes wrong.

## Expected targets

DEFRA 6 pollutants across ~18 stations:

| Pollutant | Expected stations |
| --- | --- |
| NO2 | ~15 |
| PM10 | ~8 |
| PM2.5 | ~6 |
| O3 | ~5 |
| SO2 | ~2 |
| CO | ~1 |

Total: approximately 35-50 station-pollutant combinations (varies based on data availability after filtering).

## Output path

Results saved to: `data/defra/rf_model_all/`

In [ ]:
# mandatory libraries
import numpy as np
import pandas as pd
import joblib
import os
import gc
import time
from pathlib import Path

# scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# visualisation
import matplotlib.pyplot as plt

# for progress tracking
from datetime import datetime

## File paths

Loading from the ml_prep_all output folder where the 6-pollutant prepared arrays are saved.

In [ ]:
# paths setup
base_dir = Path.cwd().parent.parent / "data" / "defra"
ml_prep_dir = base_dir / "ml_prep_all"

# output folder
rf_output_dir = base_dir / "rf_model_all"
rf_output_dir.mkdir(parents=True, exist_ok=True)

# checkpoint folder
checkpoint_dir = rf_output_dir / "checkpoints"
checkpoint_dir.mkdir(parents=True, exist_ok=True)

print(f"Loading data from: {ml_prep_dir}")
print(f"Saving results to: {rf_output_dir}")
print(f"Checkpoints: {checkpoint_dir}")

## 1) Load prepared data

In [ ]:
print("Loading data...")

X_train_rf = np.load(ml_prep_dir / "X_train_rf.npy")
X_val_rf = np.load(ml_prep_dir / "X_val_rf.npy")
X_test_rf = np.load(ml_prep_dir / "X_test_rf.npy")

y_train = np.load(ml_prep_dir / "y_train.npy")
y_val = np.load(ml_prep_dir / "y_val.npy")
y_test = np.load(ml_prep_dir / "y_test.npy")

feature_names = joblib.load(ml_prep_dir / "feature_names.joblib")
rf_feature_names = joblib.load(ml_prep_dir / "rf_feature_names.joblib")
scaler = joblib.load(ml_prep_dir / "scaler.joblib")

print(f"\nData loaded:")
print(f"  X_train_rf: {X_train_rf.shape}")
print(f"  X_val_rf: {X_val_rf.shape}")
print(f"  X_test_rf: {X_test_rf.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  Features: {len(feature_names)}")

## 2) Identify pollutant targets

The y array contains all features including temporal columns. We need to separate the actual pollutant targets from temporal features (hour, day_of_week, month, is_weekend).

In [ ]:
# separate pollutant targets from temporal columns
temporal_cols = ['hour', 'day_of_week', 'month', 'is_weekend']

pollutant_indices = []
pollutant_names = []

for i, name in enumerate(feature_names):
    if name not in temporal_cols:
        pollutant_indices.append(i)
        pollutant_names.append(name)

print(f"Total features: {len(feature_names)}")
print(f"Temporal features: {len(temporal_cols)}")
print(f"Pollutant targets: {len(pollutant_names)}")

# count by pollutant type
print(f"\nTargets by pollutant type:")
for poll in ['NO2', 'PM25', 'PM10', 'O3', 'SO2', 'CO']:
    count = len([n for n in pollutant_names if poll in n])
    print(f"  {poll}: {count} stations")

In [ ]:
# list all targets
print("All pollutant targets:")
for i, name in enumerate(pollutant_names):
    print(f"  {i:2d}: {name}")

## 3) Define training parameters

Using memory-safe parameters optimised for 8GB RAM Mac. These were tuned in earlier single-station experiments.

### Why these parameters?

| Parameter | Value | Reasoning |
| --- | --- | --- |
| n_estimators | 100 | Balance between accuracy and memory |
| max_depth | 15 | Limits tree size to prevent memory overflow |
| min_samples_leaf | 2 | From hyperparameter tuning |
| min_samples_split | 5 | From hyperparameter tuning |
| n_jobs | 1 | Single core to prevent memory spikes |

Using `n_jobs=1` instead of `-1` (all cores) because parallel tree building multiplies memory usage. For overnight training, stability matters more than speed.

In [ ]:
# memory-safe RF parameters
RF_PARAMS = {
    'n_estimators': 100,
    'max_depth': 15,
    'min_samples_leaf': 2,
    'min_samples_split': 5,
    'random_state': 42,
    'n_jobs': 1  # single core for memory safety
}

# checkpoint frequency
CHECKPOINT_EVERY = 10  # save progress every 10 models

print("RF Parameters:")
for param, value in RF_PARAMS.items():
    print(f"  {param}: {value}")
print(f"\nCheckpoint every {CHECKPOINT_EVERY} models")

## 4) Define evaluation function

In [ ]:
def evaluate_model(model, X, y_true):
    """
    Evaluate model using RMSE, MAE, R².
    Returns dict with metrics.
    """
    y_pred = model.predict(X)
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'y_pred': y_pred
    }

## 5) Train models for all targets

This is the main training loop. It trains one RF model per target and saves checkpoints periodically.

### Checkpoint strategy

Every 10 models, the current results are saved to a checkpoint file. If training is interrupted, you can resume from the last checkpoint.

### Memory management

After training each model, garbage collection runs to free memory. This prevents gradual memory buildup that could crash overnight training.

### Estimated time

With ~40 targets and ~2-3 minutes per model: approximately 1.5-2 hours total.

In [ ]:
# check for existing checkpoint
checkpoint_file = checkpoint_dir / "training_checkpoint.joblib"
results_so_far = []
start_idx = 0

if checkpoint_file.exists():
    checkpoint_data = joblib.load(checkpoint_file)
    results_so_far = checkpoint_data['results']
    start_idx = checkpoint_data['next_idx']
    print(f"Resuming from checkpoint: {start_idx}/{len(pollutant_names)} models completed")
else:
    print("Starting fresh training")

print(f"\nTargets to train: {len(pollutant_names) - start_idx}")

In [ ]:
print("=" * 60)
print("RANDOM FOREST TRAINING - DEFRA ALL STATIONS (6 POLLUTANTS)")
print("=" * 60)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Targets: {len(pollutant_names)}")
print(f"Training samples: {X_train_rf.shape[0]:,}")
print(f"Features: {X_train_rf.shape[1]:,}")
print("=" * 60)

all_results = results_so_far.copy()
total_start = time.time()

for idx in range(start_idx, len(pollutant_names)):
    target_name = pollutant_names[idx]
    target_idx = pollutant_indices[idx]
    
    # extract single target
    y_train_single = y_train[:, target_idx]
    y_val_single = y_val[:, target_idx]
    y_test_single = y_test[:, target_idx]
    
    # train model
    print(f"\n[{idx+1}/{len(pollutant_names)}] Training: {target_name}")
    model_start = time.time()
    
    rf = RandomForestRegressor(**RF_PARAMS)
    rf.fit(X_train_rf, y_train_single)
    
    train_time = time.time() - model_start
    
    # evaluate
    train_metrics = evaluate_model(rf, X_train_rf, y_train_single)
    val_metrics = evaluate_model(rf, X_val_rf, y_val_single)
    test_metrics = evaluate_model(rf, X_test_rf, y_test_single)
    
    print(f"  Time: {train_time:.1f}s | Test R²: {test_metrics['r2']:.4f} | RMSE: {test_metrics['rmse']:.4f}")
    
    # store results
    result = {
        'target': target_name,
        'target_idx': target_idx,
        'train_time': train_time,
        'train_rmse': train_metrics['rmse'],
        'train_mae': train_metrics['mae'],
        'train_r2': train_metrics['r2'],
        'val_rmse': val_metrics['rmse'],
        'val_mae': val_metrics['mae'],
        'val_r2': val_metrics['r2'],
        'test_rmse': test_metrics['rmse'],
        'test_mae': test_metrics['mae'],
        'test_r2': test_metrics['r2']
    }
    all_results.append(result)
    
    # save model
    model_file = rf_output_dir / f"rf_model_{target_name}.joblib"
    joblib.dump(rf, model_file)
    
    # save predictions
    predictions = {
        'y_train_actual': y_train_single,
        'y_train_pred': train_metrics['y_pred'],
        'y_val_actual': y_val_single,
        'y_val_pred': val_metrics['y_pred'],
        'y_test_actual': y_test_single,
        'y_test_pred': test_metrics['y_pred']
    }
    joblib.dump(predictions, rf_output_dir / f"predictions_{target_name}.joblib")
    
    # checkpoint
    if (idx + 1) % CHECKPOINT_EVERY == 0:
        checkpoint_data = {
            'results': all_results,
            'next_idx': idx + 1,
            'timestamp': datetime.now().isoformat()
        }
        joblib.dump(checkpoint_data, checkpoint_file)
        print(f"  [Checkpoint saved: {idx+1} models complete]")
    
    # memory cleanup
    del rf, train_metrics, val_metrics, test_metrics
    gc.collect()

total_time = time.time() - total_start
print("\n" + "=" * 60)
print(f"TRAINING COMPLETE")
print(f"Total time: {total_time/60:.1f} minutes")
print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

## 6) Results summary

In [ ]:
# create results dataframe
results_df = pd.DataFrame(all_results)

# add pollutant type column
def get_pollutant_type(name):
    for poll in ['NO2', 'PM25', 'PM10', 'O3', 'SO2', 'CO']:
        if poll in name:
            return poll
    return 'Other'

results_df['pollutant'] = results_df['target'].apply(get_pollutant_type)

# save results
results_df.to_csv(rf_output_dir / 'all_results.csv', index=False)
print(f"Results saved to: {rf_output_dir / 'all_results.csv'}")

results_df.head(10)

In [ ]:
# overall performance
print("Overall Performance Summary")
print("=" * 50)
print(f"\nTotal targets trained: {len(results_df)}")
print(f"Total training time: {results_df['train_time'].sum()/60:.1f} minutes")

print(f"\nTest Set Metrics (mean across all targets):")
print(f"  RMSE: {results_df['test_rmse'].mean():.4f} (± {results_df['test_rmse'].std():.4f})")
print(f"  MAE:  {results_df['test_mae'].mean():.4f} (± {results_df['test_mae'].std():.4f})")
print(f"  R²:   {results_df['test_r2'].mean():.4f} (± {results_df['test_r2'].std():.4f})")

In [ ]:
# performance by pollutant type
print("Performance by Pollutant Type")
print("=" * 50)

pollutant_summary = results_df.groupby('pollutant').agg({
    'test_rmse': ['mean', 'std', 'count'],
    'test_mae': 'mean',
    'test_r2': 'mean'
}).round(4)

pollutant_summary.columns = ['RMSE_mean', 'RMSE_std', 'n_stations', 'MAE_mean', 'R2_mean']
pollutant_summary = pollutant_summary.sort_values('R2_mean', ascending=False)

print(pollutant_summary)

In [ ]:
# best and worst performing targets
print("\nTop 5 Best Performing Targets (by R²):")
print("-" * 50)
best = results_df.nlargest(5, 'test_r2')[['target', 'test_r2', 'test_rmse', 'test_mae']]
print(best.to_string(index=False))

print("\nTop 5 Worst Performing Targets (by R²):")
print("-" * 50)
worst = results_df.nsmallest(5, 'test_r2')[['target', 'test_r2', 'test_rmse', 'test_mae']]
print(worst.to_string(index=False))

## 7) Visualisation

In [ ]:
# R² distribution by pollutant
fig, ax = plt.subplots(figsize=(10, 6))

pollutants = ['NO2', 'PM10', 'PM25', 'O3', 'SO2', 'CO']
colors = ['steelblue', 'coral', 'seagreen', 'gold', 'purple', 'brown']

data_to_plot = []
labels = []
for poll in pollutants:
    poll_data = results_df[results_df['pollutant'] == poll]['test_r2']
    if len(poll_data) > 0:
        data_to_plot.append(poll_data.values)
        labels.append(f"{poll}\n(n={len(poll_data)})")

bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True)
for patch, color in zip(bp['boxes'], colors[:len(data_to_plot)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax.set_ylabel('Test R²')
ax.set_title('DEFRA Random Forest Performance by Pollutant Type')
ax.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='R²=0.8 threshold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(rf_output_dir / 'r2_by_pollutant.png', dpi=150)
plt.show()

In [ ]:
# scatter plot: actual vs predicted for best model
best_target = results_df.loc[results_df['test_r2'].idxmax(), 'target']
best_preds = joblib.load(rf_output_dir / f"predictions_{best_target}.joblib")

fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(best_preds['y_test_actual'], best_preds['y_test_pred'], 
           alpha=0.3, s=10, c='steelblue')

# perfect prediction line
lims = [0, 1]
ax.plot(lims, lims, 'r--', alpha=0.8, label='Perfect prediction')

ax.set_xlabel('Actual (normalised)')
ax.set_ylabel('Predicted (normalised)')
ax.set_title(f'Best Model: {best_target}')
ax.legend()
ax.set_xlim(lims)
ax.set_ylim(lims)

plt.tight_layout()
plt.savefig(rf_output_dir / 'best_model_scatter.png', dpi=150)
plt.show()

## 8) Training summary

In [ ]:
print("=" * 60)
print("DEFRA RANDOM FOREST TRAINING SUMMARY (6 POLLUTANTS)")
print("=" * 60)

print(f"\nDataset:")
print(f"  Training samples: {X_train_rf.shape[0]:,}")
print(f"  Validation samples: {X_val_rf.shape[0]:,}")
print(f"  Test samples: {X_test_rf.shape[0]:,}")
print(f"  Features: {X_train_rf.shape[1]:,}")

print(f"\nModels trained: {len(results_df)}")
print(f"Total training time: {results_df['train_time'].sum()/60:.1f} minutes")

print(f"\nOverall Test Performance:")
print(f"  Mean R²:   {results_df['test_r2'].mean():.4f}")
print(f"  Mean RMSE: {results_df['test_rmse'].mean():.4f}")
print(f"  Mean MAE:  {results_df['test_mae'].mean():.4f}")

print(f"\nBest model: {best_target}")
print(f"  R²: {results_df['test_r2'].max():.4f}")

print(f"\nFiles saved to: {rf_output_dir}")
print("=" * 60)

## 9) Cleanup checkpoint (optional)

Run this cell after training completes successfully to remove the checkpoint file.

In [ ]:
# remove checkpoint after successful completion
if checkpoint_file.exists():
    checkpoint_file.unlink()
    print("Checkpoint file removed.")
else:
    print("No checkpoint file to remove.")

## Next steps

1. Compare these results with LAQN all-stations RF to see dataset differences.
2. Run CNN training on the same targets for model comparison.
3. Analyse which stations/pollutants perform best and why.
4. Create comprehensive results tables for dissertation.